 Install the **pandas-gbq** library, which is a Pandas-based interface for working with Google BigQuery and other dependencies libraries.

In [ ]:
!pip install pandas-gbq
!pip install pydata_google_auth
!pip install --upgrade google-cloud-bigquery google-cloud-bigquery-storage

To avoid printing any warning message on console

In [2]:
import warnings

warnings.filterwarnings("ignore", category=UserWarning, module="google.cloud.bigquery._pandas_helpers")


Get GCP credentials and interact with Bigquery

In [1]:
import pydata_google_auth
credentials = pydata_google_auth.get_user_credentials(['https://www.googleapis.com/auth/bigquery'],)

In [3]:
project_id = 'silicon-parity-396203'

In [5]:
import pandas as pd
pd.set_option('display.max_colwidth', None)

In [23]:
!bq load --source_format=NEWLINE_DELIMITED_JSON mydataset.table2 ip_file.jsonl name:STRING,cart:JSON
!bq show mydataset.table2

BigQuery error in load operation: Provided Schema does not match Table silicon-
parity-396203:mydataset.table2. Cannot add fields (field: cart)
Table silicon-parity-396203:mydataset.table2

   Last modified        Schema        Total Rows   Total Bytes   Expiration   Time Partitioning   Clustered Fields   Total Logical Bytes   Total Physical Bytes   Labels  
 ----------------- ----------------- ------------ ------------- ------------ ------------------- ------------------ --------------------- ---------------------- -------- 
  18 Sep 11:48:39   |- name: string   3            227                                                               227                   1243                           
                    |- items: json                                                                                                                                        



**How to Query JSON data**

In [24]:
query = """select * from mydataset.table2"""

df = pd.read_gbq(query, project_id=project_id, dialect='standard', credentials=credentials)
df

,name,items
0,Alice,"[{""price"":10,""product"":""book""},{""price"":5,""product"":""food""}]"
1,Bob,"[{""price"":2,""product"":""pen""},{""price"":20,""product"":""clothing""}]"
2,Charlie,"[{""price"":1000,""product"":""computer""},{""price"":50,""product"":""electronics""}]"


Access the fields in a JSON expression by using the field access operator.

To access an array element, use the JSON subscript operator

In [27]:
query = """SELECT name,items[0] as first_item FROM mydataset.table2;"""
df = pd.read_gbq(query, project_id=project_id, dialect='standard', credentials=credentials)
df

,name,first_item
0,Alice,"{""price"":10,""product"":""book""}"
1,Bob,"{""price"":2,""product"":""pen""}"
2,Charlie,"{""price"":1000,""product"":""computer""}"


In [28]:
query = """SELECT name,items[0]['product'] as first_item FROM mydataset.table2;"""
df = pd.read_gbq(query, project_id=project_id, dialect='standard', credentials=credentials)
df

,name,first_item
0,Alice,"""book"""
1,Bob,"""pen"""
2,Charlie,"""computer"""


JSON_QUERY: Extracts a JSON value, such as an array or object, or a scalar JSON-formatted value, such as a string, number, or boolean value.

JSON_VALUE Extracts a scalar value. A scalar value can represent a string, a number, or a boolean value.

JSON_EXTRACT: Extracts a JSON value, such as an array or object, or a scalar JSON-formatted value, such as a string, number, or boolean value.

PARSE_JSON: Returns a JSON value for a JSON formatted string.

TO_JSON: Returns a JSON value for an SQL value.

TO_JSON_STRING: Accepts an SQL value and returns a JSON formatted string representation of the value.

In [31]:
query = """SELECT name,items[0] as first_item FROM mydataset.table2
where JSON_VALUE(items[0]['product']) = 'book';"""
df = pd.read_gbq(query, project_id=project_id, dialect='standard', credentials=credentials)
df

,name,first_item
0,Alice,"{""price"":10,""product"":""book""}"


JSON_QUERY_ARRAY: extracts an array and returns it as an ARRAY<JSON> of JSON.

JSON_VALUE_ARRAY: extracts an array of scalar values and returns it as an ARRAY<STRING> of scalar values.

In [37]:
query = """SELECT name,JSON_QUERY_ARRAY(items) as first_item FROM mydataset.table2;"""
df = pd.read_gbq(query, project_id=project_id, dialect='standard', credentials=credentials)
df

,name,first_item
0,Alice,"[{""price"":10,""product"":""book""}, {""price"":5,""product"":""food""}]"
1,Bob,"[{""price"":2,""product"":""pen""}, {""price"":20,""product"":""clothing""}]"
2,Charlie,"[{""price"":1000,""product"":""computer""}, {""price"":50,""product"":""electronics""}]"


In [43]:
query = """
SELECT name, ARRAY_AGG(JSON_VALUE(item.product)) as all_product, ARRAY_AGG(JSON_VALUE(item.price)) as all_price 
FROM mydataset.table2 ,UNNEST(JSON_QUERY_ARRAY(items)) AS item
group by name
order by name
;"""
df = pd.read_gbq(query, project_id=project_id, dialect='standard', credentials=credentials)
df

,name,all_product,all_price
0,Alice,"[book, food]","[10, 5]"
1,Bob,"[pen, clothing]","[2, 20]"
2,Charlie,"[computer, electronics]","[1000, 50]"


In [51]:
query = """SELECT items[0].product , JSON_QUERY(items[0],'$.product') as json_price ,JSON_VALUE(items[0],'$.product') as json_value_price FROM mydataset.table2;"""
df = pd.read_gbq(query, project_id=project_id, dialect='standard', credentials=credentials)
df

,product,json_price,json_value_price
0,"""book""","""book""",book
1,"""pen""","""pen""",pen
2,"""computer""","""computer""",computer


In [52]:
query = """CREATE or Replace TABLE mydataset.table1(
  id INT64,
  cart JSON
)"""

df = pd.read_gbq(query, project_id=project_id, dialect='standard', credentials=credentials)
df

,id,cart


You can't partition or cluster a table on JSON columns

![Alt text](image.png)

In [53]:
query = """INSERT INTO mydataset.table1 VALUES
(1, JSON '{"name": "Alice", "age": 30}'),
(2, JSON_ARRAY(10, ['foo', 'bar'], [20, 30])),
(3, JSON_OBJECT('foo', 10, 'bar', ['a', 'b']))"""
df = pd.read_gbq(query, project_id=project_id, dialect='standard', credentials=credentials)
df

,id,cart
0,3,"{""bar"":[""a"",""b""],""foo"":10}"
1,2,"[10,[""foo"",""bar""],[20,30]]"
2,1,"{""age"":30,""name"":""Alice""}"


In [54]:
query = """WITH Fruits AS (
SELECT 0 AS id, 'color' AS k, 'Red' AS v UNION ALL
SELECT 0, 'fruit', 'apple' UNION ALL
SELECT 1, 'fruit','banana' UNION ALL
SELECT 1, 'ripe', 'true'
)

SELECT JSON_OBJECT(ARRAY_AGG(k), ARRAY_AGG(v)) AS json_data
FROM Fruits
GROUP BY id
"""
df = pd.read_gbq(query, project_id=project_id, dialect='standard', credentials=credentials)
df

,json_data
0,"{""color"":""Red"",""fruit"":""apple""}"
1,"{""fruit"":""banana"",""ripe"":""true""}"


In [56]:
query = """CREATE or Replace TABLE mydataset.table1(
  id INT64,
  json_data JSON
)"""
df = pd.read_gbq(query, project_id=project_id, dialect='standard', credentials=credentials)
df

,id,json_data


In [57]:
!bq load --source_format=CSV mydataset.table1 input.csv id:INTEGER,json_data:JSON


Upload complete.

Waiting on bqjob_r32b8736bed5caa92_0000018aa7f02ed6_1 ... (0s) Current status: RUNNING
                                                                                      
Waiting on bqjob_r32b8736bed5caa92_0000018aa7f02ed6_1 ... (0s) Current status: DONE   


![Alt text](image-1.png)

In [62]:
query = """WITH A AS (
  SELECT JSON '{"name": "Alice", "age": 28}' AS user UNION ALL
  SELECT JSON '{"name": "Bob", "age": 40 }' UNION ALL
  SELECT JSON '{"name": "Avery", "age": null}'
)

-- Table contents
SELECT user.name, user.age AS json_navigation_to_age,
-- INT64(user.age) AS int64_user_age,
SAFE.INT64(user.age) AS safe_int64_user_age,
LAX_INT64(user.age) AS lax_user_age
FROM A;"""
df = pd.read_gbq(query, project_id=project_id, dialect='standard', credentials=credentials)
df

,name,json_navigation_to_age,safe_int64_user_age,lax_user_age
0,"""Alice""",28,28,28
1,"""Bob""",40,40,40
2,"""Avery""",null,<NA>,<NA>
